In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('./data/sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
training_summary = lrModel.summary
training_summary

In [11]:
training_summary.r2

0.027839179518600154

In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

In [13]:
all_data = spark.read.format('libsvm').load('./data/sample_linear_regression_data.txt')

In [14]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [15]:
train_data, test_data

(DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector])

In [16]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
+----------

In [17]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                161|
|   mean| 1.2832372306583473|
| stddev| 10.117620640304438|
|    min|-26.805483428483072|
|    max|  27.78383192005107|
+-------+-------------------+



In [18]:
correct_model = lr.fit(train_data)

In [19]:
test_results = correct_model.evaluate(test_data)

In [20]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-27.222522276179852|
| -20.42050864192216|
| -21.50727510797742|
| -21.74544232976524|
|-19.596340218003657|
| -17.62664191975894|
|-16.853404075208342|
|-18.399131990577335|
|-14.597554791706145|
| -13.49061563497698|
| -16.47100730071679|
|-15.138920578884871|
|-12.854500596752667|
|-17.601183431844113|
|-14.961685414016614|
|-16.800053598479025|
|-14.989851172109026|
| -16.00565038190166|
|-13.802166539732971|
|-10.607342197207991|
+-------------------+
only showing top 20 rows



In [21]:
test_results.rootMeanSquaredError

10.23583825209279

In [22]:
unlabeled_data = test_data.select('features')

In [23]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [24]:
predictions = correct_model.transform(unlabeled_data)

In [25]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  0.4170388476967801|
|(10,[0,1,2,3,4,5,...| -2.4169517749971803|
|(10,[0,1,2,3,4,5,...|   0.074887343811613|
|(10,[0,1,2,3,4,5,...|  1.5333650708065691|
|(10,[0,1,2,3,4,5,...| -0.2766508200647476|
|(10,[0,1,2,3,4,5,...| -1.5416507032048214|
|(10,[0,1,2,3,4,5,...| -0.6407962816750017|
|(10,[0,1,2,3,4,5,...|  0.9704574196378273|
|(10,[0,1,2,3,4,5,...|  -2.428937472503403|
|(10,[0,1,2,3,4,5,...| -2.5950434060445104|
|(10,[0,1,2,3,4,5,...|  0.6089979731462273|
|(10,[0,1,2,3,4,5,...| -0.5931676933543739|
|(10,[0,1,2,3,4,5,...| -2.5828841966785507|
|(10,[0,1,2,3,4,5,...|   2.266415951921772|
|(10,[0,1,2,3,4,5,...|-0.09479756052581867|
|(10,[0,1,2,3,4,5,...|   3.027612036776155|
|(10,[0,1,2,3,4,5,...|   2.012002446716922|
|(10,[0,1,2,3,4,5,...|    3.08342727853124|
|(10,[0,1,2,3,4,5,...|  1.3107244621865575|
|(10,[0,1,2,3,4,5,...|  -1.52301